In [ ]:
!pip install ninja

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
import torch
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
sys.path.append('/content/Self-Correction-Human-Parsing')
import networks
from utils.transforms import transform_logits
from datasets.simple_extractor_dataset import SimpleFolderDataset

In [ ]:
# import os

# init_file = "/content/Self-Correction-Human-Parsing/networks/__init__.py"
# if not os.path.exists(init_file):
#     open(init_file, 'w').close()
# print(" __init__.py")


In [ ]:
# pth_path = "/content/drive/MyDrive/singleimg models/exp-schp-201908261155-lip.pth"
pth_path = "/content/drive/MyDrive/singleimg models/exp-schp-201908270938-pascal-person-part.pth"

checkpoint = torch.load(pth_path)
print(checkpoint.keys())


In [ ]:
def get_palette(num_cls):
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


In [ ]:

input_dir = "/content/drive/MyDrive/Image_train"
img_name = "001186_M_28_175260_5896701.png"

dataset_name = "pascal"
dataset_settings = {
    'lip': {
        'input_size': [473, 473],
        'num_classes': 20,
        'label': ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses',
                  'Upper-clothes', 'Dress', 'Coat', 'Socks', 'Pants',
                  'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm',
                  'Right-arm', 'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
    },
    'pascal': {
        'input_size': [512, 512],
        'num_classes': 7,
        'label': ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs'],
    }
}

input_size = dataset_settings[dataset_name]['input_size']
num_classes = dataset_settings[dataset_name]['num_classes']
label_list = dataset_settings[dataset_name]['label']


print("construct...")
model = networks.init_model('resnet101', num_classes=num_classes, pretrained=None)

state_dict = torch.load(pth_path, map_location="cpu")['state_dict']
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] if k.startswith("module.") else k
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model.eval()
model.cuda()
print("loaded ")


In [ ]:
class HumanParser(object):
    def __init__(self, model = model, input_sizee = [473, 473], num_classes = 20):
        self.model = torch.nn.DataParallel(model).cuda()
        self.model.eval()
        self.input_size = input_size
        self.num_classes = num_classes
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.406, 0.456, 0.485],
                                 std=[0.225, 0.224, 0.229])
        ])

    def Arms_detect(self, img):
        # print("debug!")
        # print("Arms_detect received:", type(img))
        # if isinstance(img, np.ndarray):
        #    img = torch.from_numpy(img)
        h, w, _ = img.shape
        aspect_ratio = self.input_size[1] * 1.0 / self.input_size[0]
        person_center, s = self.box2cs([0, 0, w - 1, h - 1], aspect_ratio)
        r = 0
        trans = self.get_affine_transform(person_center, s, r, self.input_size)
        warped = cv2.warpAffine(
            img,
            trans,
            (int(self.input_size[1]), int(self.input_size[0])),
            flags=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(0, 0, 0)
        )
        warped = cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)
        input_tensor = self.transform(warped).unsqueeze(dim=0)
        output = self.model(input_tensor.cuda())
        # print([type(o) for o in output], [[o.shape if isinstance(o, torch.Tensor) else "list" for o in group] for group in output])

        # parsing_result = output[0][0]
        # plt.imshow(parsing_result.detach().cpu().numpy()[0], cmap="gray")
        # plt.title("Parsing Result1")
        # plt.axis("off")
        # plt.show()
        parsing_result = output[0][1]
        # plt.imshow(parsing_result.detach().cpu().numpy()[0], cmap="gray")
        # plt.title("Parsing Result2")
        # plt.axis("off")
        # plt.show()

        if not isinstance(parsing_result, torch.Tensor):
            raise TypeError(f"Expected Tensor, but got {type(parsing_result)}")
        upsample_output = torch.nn.functional.interpolate(parsing_result, size=self.input_size, mode='bilinear', align_corners=True)
        # upsample_output = upsample(output)
        upsample_output = upsample_output.squeeze()
        upsample_output = upsample_output.permute(1, 2, 0)  # CHW -> HWC
        upsample_output = upsample_output.data.cpu().numpy()
        trans = self.get_affine_transform(person_center, s, 0, self.input_size, inv=1)
        channel = upsample_output.shape[2]
        target_logits = []
        for i in range(channel):
            target_logit = cv2.warpAffine(
                upsample_output[:, :, i],
                trans,
                (int(w), int(h)),  # (int(width), int(height)),
                flags=cv2.INTER_LINEAR,
                borderMode=cv2.BORDER_CONSTANT,
                borderValue=(0)
            )
            target_logits.append(target_logit)
        target_logits = np.stack(target_logits, axis=2)
        parsing_result = np.argmax(target_logits, axis=2)
        # result = list(map(lambda x: list(map(lambda y: y == 14 or y == 15, x)), parsing_result))
        result = list(map(lambda x: list(map(lambda y: y == 3 or y == 4, x)), parsing_result))
        result = np.asarray(result)
        # print(result.shape)
        return result

    def get_3rd_point(self, a, b):
        direct = a - b
        return b + np.array([-direct[1], direct[0]], dtype=np.float32)

    def get_dir(self, src_point, rot_rad):
        sn, cs = np.sin(rot_rad), np.cos(rot_rad)
        src_result = [src_point[0] * cs - src_point[1] * sn,
                      src_point[0] * sn + src_point[1] * cs]
        return src_result

    def get_affine_transform(self, center, scale, rot, output_size, shift=np.array([0, 0], dtype=np.float32), inv=0):
        if not isinstance(scale, np.ndarray) and not isinstance(scale, list):
            scale = np.array([scale, scale], dtype=np.float32)
        src_w = scale[0]
        dst_w = output_size[1]
        dst_h = output_size[0]
        rot_rad = np.pi * rot / 180
        src_dir = self.get_dir([0, src_w * -0.5], rot_rad)
        dst_dir = np.array([0, (dst_w - 1) * -0.5], np.float32)
        src = np.zeros((3, 2), dtype=np.float32)
        dst = np.zeros((3, 2), dtype=np.float32)
        src[0, :] = center + scale * shift
        src[1, :] = center + src_dir + scale * shift
        dst[0, :] = [ (dst_w - 1) * 0.5, (dst_h - 1) * 0.5 ]
        dst[1, :] = [ (dst_w - 1) * 0.5, (dst_h - 1) * 0.5 ] + dst_dir
        src[2, :] = self.get_3rd_point(src[0, :], src[1, :])
        dst[2, :] = self.get_3rd_point(dst[0, :], dst[1, :])
        if inv:
            trans = cv2.getAffineTransform(np.float32(dst), np.float32(src))
        else:
            trans = cv2.getAffineTransform(np.float32(src), np.float32(dst))
        return trans

    def box2cs(self, box, ar):
        x, y, w, h = box[:4]
        return self.xywh2cs(x, y, w, h, ar)

    def xywh2cs(self, x, y, w, h, ar):
        center = np.zeros(2, dtype=np.float32)
        center[0] = x + w * 0.5
        center[1] = y + h * 0.5
        if w > ar * h:
            h = w * 1.0 / ar
        elif w < ar * h:
            w = h * ar
        scale = np.array([w, h], dtype=np.float32)
        return center, scale

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/singleimg models/modeling')
sys.path.append('/content/drive/MyDrive/singleimg models')
sys.path.append('/content/drive/MyDrive/singleimg models/lib')
sys.path.append('/content/drive/MyDrive/singleimg models/lib/transforms')

from build_model import Pose2Seg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

In [ ]:
import detectron2
print(detectron2.__version__)


In [ ]:
class Body_Figure(object):
    def __init__(self, waist_width, thigh_width, hip_width, head_width, Area, height, shoulder_width):
        self._waist_width = waist_width
        self._thigh_width = thigh_width
        self._hip_width = hip_width
        self._head_width = head_width
        self._Area = Area
        self._height = height
        self._shoulder_width = shoulder_width
        if self._head_width == 0:
            self._head_width = self._hip_width / 3

    @property
    def WSR(self):
        return self._waist_width / self._shoulder_width

    @property
    def WTR(self):
        return self._waist_width / self._thigh_width

    @property
    def WHpR(self):
        return self._waist_width / self._hip_width

    @property
    def WHdR(self):
        return self._waist_width / self._head_width

    @property
    def HpHdR(self):
        return self._hip_width / self._head_width

    @property
    def Area(self):
        return self._Area

    @property
    def H2W(self):
        return self._height / self._waist_width


class Image_Processor(object):

    def __init__(self, masks_file, key_file, key_thresh=0.7):
        self._KeypointCfg = self.__init_key(key_file, key_thresh)
        self._ContourPredictor = self.__init_mask(masks_file)
        self._KeypointsPredictor = DefaultPredictor(self._KeypointCfg)
        self._HumanParser = HumanParser()
        print("check")

    def __init_mask_RCNN(self):
        cfg = get_cfg()

        cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        return cfg

    def __init_mask(self, masks_file):
        Model = Pose2Seg().cuda()
        Model.init(masks_file)
        Model.eval()
        return Model
    # num not match wtf
    # def __init_key(self, key_file, key_thresh):
    #     cfg = get_cfg()
    #     cfg.MODEL.DEVICE = "cuda"
    #     cfg.merge_from_file(model_zoo.get_config_file(key_file))
    #     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = key_thresh
    #     cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(key_file)
    #     return cfg
    def __init_key(self, key_file, key_thresh):
      cfg = get_cfg()
      cfg.merge_from_file(model_zoo.get_config_file(key_file))
      cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = key_thresh
      cfg.MODEL.DEVICE = "cuda"
      cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(key_file)

      model = DefaultPredictor(cfg).model
      detectron2.checkpoint.DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

      return cfg

    def _detected(self, img):
        # Mask RCNN
        KeypointsOutput = self._KeypointsPredictor(img)
        sorted_idxs = np.argsort(-KeypointsOutput["instances"].scores.cpu().numpy())
        Keypoints = KeypointsOutput["instances"].pred_keypoints[sorted_idxs[0]].cpu().numpy()
        gt_kpts = Keypoints[None, :, :]

        # Pose2Seg
        ContourOutput = self._ContourPredictor([img], [gt_kpts])
        ContourOutput = np.squeeze(np.asarray(ContourOutput))
        # plt.figure(figsize=(10,5))

        # plt.subplot(1,2,1)
        # plt.imshow(ContourOutput, cmap="gray")
        # plt.title("before remove arms")


        Arms_mask = self._HumanParser.Arms_detect(img)
        ContourOutput = ContourOutput ^ Arms_mask
        # plt.subplot(1,2,2)
        # plt.imshow(ContourOutput, cmap="gray")
        # plt.title("after remove arms")

        # plt.show()
        return Keypoints, ContourOutput

    def Process(self, img_RGB):
        img_keypoints, img_mask = self._detected(img_RGB)
        nose, left_eye, right_eye, left_ear, right_ear = img_keypoints[0], img_keypoints[1], img_keypoints[2], img_keypoints[3], img_keypoints[4]
        left_shoulder, right_shoulder = img_keypoints[5], img_keypoints[6]
        left_elbow, right_elbow = img_keypoints[7], img_keypoints[8]
        left_wrist, right_wrist = img_keypoints[9], img_keypoints[10]
        left_hip, right_hip = img_keypoints[11], img_keypoints[12]
        left_knee, right_knee = img_keypoints[13], img_keypoints[14]
        left_ankle, right_ankle = img_keypoints[15], img_keypoints[16]


        y_hip = (left_hip[1] + right_hip[1]) / 2
        y_knee = (left_knee[1] + right_knee[1]) / 2

        center_shoulder = (left_shoulder + right_shoulder) / 2
        y_waist = y_hip * 2 / 3 + (nose[1] + center_shoulder[1]) / 6
        left_thigh = (left_knee + left_hip) / 2
        right_thigh = (right_knee + right_hip) / 2

        waist_width = self.waist_width_estimate(center_shoulder, y_waist, img_mask)
        thigh_width = self.thigh_width_estimate(left_thigh, right_thigh, img_mask)
        hip_width = self.hip_width_estimate(center_shoulder, y_hip, img_mask)
        head_width = self.head_width_estimate(left_ear, right_ear)
        Area = self.Area_estimate(y_waist, y_hip, waist_width, hip_width, img_mask)
        height = self.Height_estimate(y_knee, nose[1])
        shoulder_width = self.shoulder_width_estimate(left_shoulder, right_shoulder)

        figure = Body_Figure(waist_width, thigh_width, hip_width, head_width, Area, height, shoulder_width)
        return figure

    def Height_estimate(self, y_k, y_n):
        return np.abs(y_n - y_k)

    def Area_estimate(self, y_w, y_h, W_w, H_w, mask):
        pixels = np.sum(mask[int(y_w):int(y_h)][:])
        area = (y_h - y_w) * 0.5 * (W_w + H_w)
        return pixels / area

    def shoulder_width_estimate(self, left_shoulder, right_shoulder):
        return np.sqrt((right_shoulder[0] - left_shoulder[0]) ** 2 + (right_shoulder[1] - left_shoulder[1]) ** 2)

    def head_width_estimate(self, left_ear, right_ear):
        return np.sqrt((right_ear[0] - left_ear[0]) ** 2 + (right_ear[1] - left_ear[1]) ** 2)

    def hip_width_estimate(self, center_shoulder, y_hip, img_mask):
        x_hip_center = int(center_shoulder[0])
        x_lhb = np.where(img_mask[int(y_hip)][:x_hip_center] == 0)[0]
        x_lhb = x_lhb[-1] if len(x_lhb) else 0
        x_rhb = np.where(img_mask[int(y_hip)][x_hip_center:] == 0)[0]
        x_rhb = x_rhb[0] + x_hip_center if len(x_rhb) else len(img_mask[0])
        return x_rhb - x_lhb

    def thigh_width_estimate(self, left_thigh, right_thigh, mask):
        lx, ly = int(left_thigh[0]), int(left_thigh[1])
        rx, ry = int(right_thigh[0]), int(right_thigh[1])
        x_ltb = np.where(mask[ly][:lx] == 0)[0]
        x_ltb = x_ltb[-1] if len(x_ltb) else 0
        x_rtb = np.where(mask[ry][rx:] == 0)[0]
        x_rtb = x_rtb[0] + rx if len(x_rtb) else len(mask[0])
        l_width = (lx - x_ltb) * 2
        r_width = (x_rtb - rx) * 2
        return (l_width + r_width) / 2

    def waist_width_estimate(self, center_shoulder, y_waist, img_mask):
        x_waist_center = int(center_shoulder[0])
        x_lwb = np.where(img_mask[int(y_waist)][:x_waist_center] == 0)[0]
        x_lwb = x_lwb[-1] if len(x_lwb) else 0
        x_rwb = np.where(img_mask[int(y_waist)][x_waist_center:] == 0)[0]
        x_rwb = x_rwb[0] + x_waist_center if len(x_rwb) else len(img_mask[0])
        return x_rwb - x_lwb

    def Vis(self, img_RGB):
      img_keypoints, img_mask = self._detected(img_RGB)

      nose, left_ear, right_ear = img_keypoints[0], img_keypoints[4], img_keypoints[3]
      left_shoulder, right_shoulder = img_keypoints[6], img_keypoints[5]
      left_hip, right_hip = img_keypoints[12], img_keypoints[11]
      left_knee, right_knee = img_keypoints[14], img_keypoints[13]

      center_shoulder = (left_shoulder + right_shoulder) / 2
      y_hip = (left_hip[1] + right_hip[1]) / 2
      y_waist = y_hip * 2 / 3 + (nose[1] + center_shoulder[1]) / 6
      left_thigh = (left_knee + left_hip) / 2
      right_thigh = (right_knee + right_hip) / 2

      img = img_RGB.copy()
      for point in [nose, left_ear, right_ear, left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee, center_shoulder, left_thigh, right_thigh]:
          cv2.circle(img, (int(point[0]), int(point[1])), 5, (0, 255, 0), -1)

      plt.figure(figsize=(6, 10))
      plt.imshow(img)
      plt.axis("off")
      plt.title("keypts vis")
      plt.show()
    def Vis_measurements(self, img_RGB):
      img_keypoints, img_mask = self._detected(img_RGB)
      plt.imshow(img_mask, cmap="gray")
      plt.title("Mask vis")
      plt.show()
      left_shoulder, right_shoulder = img_keypoints[6], img_keypoints[5]
      left_hip, right_hip = img_keypoints[12], img_keypoints[11]
      left_knee, right_knee = img_keypoints[14], img_keypoints[13]

      center_shoulder = (left_shoulder + right_shoulder) / 2
      y_hip = (left_hip[1] + right_hip[1]) / 2
      y_waist = y_hip * 2 / 3 + (img_keypoints[0][1] + center_shoulder[1]) / 6
      left_thigh = (left_knee + left_hip) / 2
      right_thigh = (right_knee + right_hip) / 2

      waist_width = self.waist_width_estimate(center_shoulder, y_waist, img_mask)
      hip_width = self.hip_width_estimate(center_shoulder, y_hip, img_mask)
      thigh_width = self.thigh_width_estimate(left_thigh, right_thigh, img_mask)

      img = img_RGB.copy()
      cv2.line(img, (int(center_shoulder[0] - waist_width / 2), int(y_waist)),
                    (int(center_shoulder[0] + waist_width / 2), int(y_waist)), (255, 0, 0), 3)
      cv2.line(img, (int(center_shoulder[0] - hip_width / 2), int(y_hip)),
                    (int(center_shoulder[0] + hip_width / 2), int(y_hip)), (0, 255, 0), 3)
      cv2.line(img, (int(left_thigh[0]), int(left_thigh[1])),
                    (int(right_thigh[0]), int(right_thigh[1])), (0, 0, 255), 3)

      plt.figure(figsize=(6, 10))
      plt.imshow(img)
      plt.axis("off")
      plt.title("wasit, hip, thigh")
      plt.show()
    def visualize_keypoints(self, img_RGB):
        #
        img_keypoints, img_mask = self._detected(img_RGB)
        nose, left_eye, right_eye, left_ear, right_ear = img_keypoints[0], img_keypoints[1], img_keypoints[2], img_keypoints[3], img_keypoints[4]
        left_shoulder, right_shoulder = img_keypoints[5], img_keypoints[6]
        left_elbow, right_elbow = img_keypoints[7], img_keypoints[8]
        left_wrist, right_wrist = img_keypoints[9], img_keypoints[10]
        left_hip, right_hip = img_keypoints[11], img_keypoints[12]
        left_knee, right_knee = img_keypoints[13], img_keypoints[14]
        left_ankle, right_ankle = img_keypoints[15], img_keypoints[16]

        img = img_RGB.copy()

        for point in img_keypoints:
            cv2.circle(img, (int(point[0]), int(point[1])), 5, (0, 255, 0), -1)

        arm_points = [left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist]
        for point in arm_points:
            cv2.circle(img, (int(point[0]), int(point[1])), 7, (255, 0, 0), -1)

        cv2.line(img, (int(left_shoulder[0]), int(left_shoulder[1])), (int(left_elbow[0]), int(left_elbow[1])), (255, 0, 0), 2)
        cv2.line(img, (int(left_elbow[0]), int(left_elbow[1])), (int(left_wrist[0]), int(left_wrist[1])), (255, 0, 0), 2)

        cv2.line(img, (int(right_shoulder[0]), int(right_shoulder[1])), (int(right_elbow[0]), int(right_elbow[1])), (255, 0, 0), 2)
        cv2.line(img, (int(right_elbow[0]), int(right_elbow[1])), (int(right_wrist[0]), int(right_wrist[1])), (255, 0, 0), 2)

        plt.figure(figsize=(6, 10))
        plt.imshow(img)
        plt.axis("off")
        plt.title("arms and other keypoints")
        plt.show()

In [ ]:
import cv2
import random

mask_model = "/content/drive/MyDrive/singleimg models/pose2seg_release.pkl"
keypoints_model = "COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"
P = Image_Processor(mask_model, keypoints_model)

In [ ]:
import traceback
# Example
# choose random image and see how those model works
image_folder = "/content/drive/MyDrive/Image_train/"

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if not image_files:
    print("No image files found in the folder!")
else:
    random_image = random.choice(image_files)
    img_path = os.path.join(image_folder, random_image)

    img_e = cv2.imread(img_path)
    if img_e is None:
        print(f"Failed to read image: {img_path}")
    else:
        print(f"Handling image: {img_path}")



        try:
            # P.Vis_measurements(img_e)
            # P.visualize_keypoints(img_e)
            F = P.Process(img_e)
            values = {
                'WSR': float(F.WSR),
                'WTR': float(F.WTR),
                'WHpR': float(F.WHpR),
                'WHdR': float(F.WHdR),
                'HpHdR': float(F.HpHdR),
                'Area': float(F.Area),
                'H2W': float(F.H2W)

            }
            print(values)
        except Exception as e:
            print(f"Error processing image: {e}")
            print(traceback.format_exc())


In [ ]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)



In [ ]:
import csv
import re

DATASET_PATHS = {
    "train": "/content/drive/MyDrive/Image_train/",
    "val": "/content/drive/MyDrive/Image_val/",
    "train_gaussian_15": "/content/drive/MyDrive/Image_train_gaussian_15/",
    "train_gaussian_30": "/content/drive/MyDrive/Image_train_gaussian_30/",
    "train_rotate_10": "/content/drive/MyDrive/Image_train_rotate_10/",
    "train_rotate_20": "/content/drive/MyDrive/Image_train_rotate_20/",
    "val_gaussian_15": "/content/drive/MyDrive/Image_val_gaussian_15/",
    "val_gaussian_30": "/content/drive/MyDrive/Image_val_gaussian_30/",
    "val_rotate_10": "/content/drive/MyDrive/Image_val_rotate_10/",
    "val_rotate_20": "/content/drive/MyDrive/Image_val_rotate_20/",
}

CSV_PATHS = {
    "train": "/content/drive/MyDrive/Image_train_features.csv",
    "val": "/content/drive/MyDrive/Image_val_features.csv",
    "train_gaussian_15": "/content/drive/MyDrive/Image_train_gaussian_15_features.csv",
    "train_gaussian_30": "/content/drive/MyDrive/Image_train_gaussian_30_features.csv",
    "train_rotate_10": "/content/drive/MyDrive/Image_train_rotate_10_features.csv",
    "train_rotate_20": "/content/drive/MyDrive/Image_train_rotate_20_features.csv",
    "val_gaussian_15": "/content/drive/MyDrive/Image_val_gaussian_15_features.csv",
    "val_gaussian_30": "/content/drive/MyDrive/Image_val_gaussian_30_features.csv",
    "val_rotate_10": "/content/drive/MyDrive/Image_val_rotate_10_features.csv",
    "val_rotate_20": "/content/drive/MyDrive/Image_val_rotate_20_features.csv",
}

def extract_features(image_path):
    img_e = cv2.imread(image_path)
    if img_e is None:
        print(f"Failed to read image: {image_path}")
        return None
    F = P.Process(img_e)
    anthropometric_features = [
        float(F.WSR), float(F.WTR), float(F.WHpR), float(F.WHdR),
        float(F.HpHdR), float(F.Area), float(F.H2W)
    ]
    return anthropometric_features

def extract_bmi_from_filename(filename):
    match = re.match(r"\d+?_([FMfm])_(\d+?)_(\d+?)_(\d+).+", filename)
    if not match:
        print(f"Skipping invalid filename: {filename}")
        return None, None, None
    try:
        height = int(match.group(3)) / 100000
        weight = int(match.group(4)) / 100000
        bmi = weight / (height ** 2)
        return height, weight, bmi
    except ValueError:
        print(f"Error parsing BMI from filename: {filename}")
        return None, None, None

BATCH_SIZE = 10

def process_dataset(dataset_type):
    image_folder = DATASET_PATHS[dataset_type]
    output_csv = CSV_PATHS[dataset_type]
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    if total_images == 0:
        print(f"No images found in {image_folder}")
        return
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        header = ["Filename", "Height", "Weight", "BMI"] + [f"Anthro_{i+1}" for i in range(7)]
        writer.writerow(header)
        for batch_start in range(0, total_images, BATCH_SIZE):
            batch_files = image_files[batch_start: batch_start + BATCH_SIZE]
            batch_results = []
            for img_name in batch_files:
                img_path = os.path.join(image_folder, img_name)
                try:
                    height, weight, bmi = extract_bmi_from_filename(img_name)
                    if height is None:
                        continue
                    features = extract_features(img_path)
                    if features is None:
                        continue
                    batch_results.append([img_name, height, weight, bmi] + features)
                except Exception as e:
                    print(f"Error processing {img_name}: {e}")
                    continue
            writer.writerows(batch_results)
            progress = min(batch_start + BATCH_SIZE, total_images)
            print(f"Processed {progress}/{total_images} images in {dataset_type} ({progress/total_images:.0%})")
    print(f"Feature extraction complete for {dataset_type}! Saved to {output_csv}")
process_dataset("train")
process_dataset("val")
process_dataset("train_gaussian_15")
process_dataset("train_gaussian_30")
process_dataset("train_rotate_10")
process_dataset("train_rotate_20")
process_dataset("val_gaussian_15")
process_dataset("val_gaussian_30")
process_dataset("val_rotate_10")
process_dataset("val_rotate_20")
